core5_01_load_all_artifacts.ipynb

목적:
- Core 2~4에서 생성된 artifact들을 한 번에 로딩
- 어떤 입력이 존재하는지 / 존재하지 않는지 명시적으로 확인
- 이후 Core 5 실패 분석이 '이 입력들에서만' 나온다는 점을 고정

중요:
- 이 노트북에서는 어떤 계산도 하지 않는다
- 수정/가공/필터링 금지

In [3]:
import json
import pandas as pd
from pathlib import Path

# Core 2
CORE2_FILES = {
    "policy_summary": Path("../artifact/core2/policy_summary.json"),
    "prediction_trace": Path("../artifact/core2/prediction_trace.csv"),
    "mutation_applied_log": Path("../artifact/core2/mutation_applied_log.csv"),
    "curieus_queue": Path("../artifact/core2/curieus_queue.csv"),
}

# Core 3
CORE3_FILES = {
    "core3_ml_dataset": Path("../artifact/core3/core3_ml_dataset.csv"),
    "instability_risk_with_trace": Path("../artifact/core3/core3_instability_risk_with_trace.csv"),
    "core3_freeze_summary": Path("../artifact/core3/core3_freeze_summary.json"),
}

# Core 4
CORE4_FILES = {
    "structural_failure_mapping": Path("../artifact/core4/core4_structural_failure_mapping.json"),
    "core4_freeze_summary": Path("../artifact/core4/core4_freeze_summary.json"),
}

def check_file_exists(path: Path):
    if path.exists():
        return True
    else:
        return False

def inspect_csv(path: Path):
    df = pd.read_csv(path)
    return {
        "rows": len(df),
        "columns": df.columns.tolist()
    }

def load_json(path: Path):
    with open(path, "r") as f:
        return json.load(f)

In [4]:
core2_status = {} # Core 2 artifact 로딩 및 상태 기록

for name, path in CORE2_FILES.items():
    if check_file_exists(path):
        if path.suffix == ".csv":
            meta = inspect_csv(path)
            core2_status[name] = {
                "exists": True,
                "type": "csv",
                "rows": meta["rows"],
                "columns": meta["columns"]
            }
            print(f"[Core2] {name}: CSV 로드 성공 ({meta['rows']} rows)")
        else:
            core2_status[name] = {
                "exists": True,
                "type": "json"
            }
            print(f"[Core2] {name}: JSON 로드 성공")
    else:
        core2_status[name] = {
            "exists": False
        }
        print(f"[Core2] {name}: ❌ 파일 없음")

[Core2] policy_summary: JSON 로드 성공
[Core2] prediction_trace: CSV 로드 성공 (60 rows)
[Core2] mutation_applied_log: CSV 로드 성공 (29 rows)
[Core2] curieus_queue: CSV 로드 성공 (29 rows)


In [5]:
core3_status = {}

for name, path in CORE3_FILES.items():
    if check_file_exists(path):
        if path.suffix == ".csv":
            meta = inspect_csv(path)
            core3_status[name] = {
                "exists": True,
                "type": "csv",
                "rows": meta["rows"],
                "columns": meta["columns"]
            }
            print(f"[Core3] {name}: CSV 로드 성공 ({meta['rows']} rows)")
        else:
            core3_status[name] = {
                "exists": True,
                "type": "json"
            }
            print(f"[Core3] {name}: JSON 로드 성공")
    else:
        core3_status[name] = {
            "exists": False
        }
        print(f"[Core3] {name}: ❌ 파일 없음") # Core 3 artifact 로딩 및 상태 기록

[Core3] core3_ml_dataset: CSV 로드 성공 (60 rows)
[Core3] instability_risk_with_trace: CSV 로드 성공 (60 rows)
[Core3] core3_freeze_summary: JSON 로드 성공


In [6]:
core4_status = {}

for name, path in CORE4_FILES.items():
    if check_file_exists(path):
        if path.suffix == ".csv":
            meta = inspect_csv(path)
            core4_status[name] = {
                "exists": True,
                "type": "csv",
                "rows": meta["rows"],
                "columns": meta["columns"]
            }
            print(f"[Core4] {name}: CSV 로드 성공 ({meta['rows']} rows)")
        else:
            core4_status[name] = {
                "exists": True,
                "type": "json"
            }
            print(f"[Core4] {name}: JSON 로드 성공")
    else:
        core4_status[name] = {
            "exists": False
        }
        print(f"[Core4] {name}: ❌ 파일 없음") # Core 4 artifact 로딩 및 상태 기록

[Core4] structural_failure_mapping: JSON 로드 성공
[Core4] core4_freeze_summary: JSON 로드 성공


In [7]:
TARGET_ANTIBODIES = [
    "GDPa1-001",
    "GDPa1-045",
    "GDPa1-183"
]

antibody_check = {}

trace_path = CORE2_FILES["prediction_trace"]

if trace_path.exists():
    trace_df = pd.read_csv(trace_path)
    for ab in TARGET_ANTIBODIES:
        antibody_check[ab] = ab in trace_df["antibody_key"].unique()
else:
    for ab in TARGET_ANTIBODIES:
        antibody_check[ab] = False

print("=== Antibody Presence Check ===")
for k, v in antibody_check.items():
    print(f"{k}: {'OK' if v else '❌ NOT FOUND'}") # 항체 3개 존재 여부 검사 (prediction_trace 기준)

=== Antibody Presence Check ===
GDPa1-001: OK
GDPa1-045: OK
GDPa1-183: OK


In [8]:
manifest = {
    "core2": core2_status,
    "core3": core3_status,
    "core4": core4_status,
    "antibody_presence": antibody_check,
    "note": "Core 5 failure analysis must rely only on the above inputs. No hidden data."
}

OUTPUT_DIR = Path("../artifact/core5")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

manifest_path = OUTPUT_DIR / "core5_inputs_manifest.json"

with open(manifest_path, "w") as f:
    json.dump(manifest, f, indent=2)

manifest_path # core5_inputs_manifest.json 생성

PosixPath('../artifact/core5/core5_inputs_manifest.json')

In [9]:
print("""
[Core 5 Input Freeze Complete]

이 노트북은 Core 2~4의 산출물을
'있는 그대로' 불러왔으며,
이후 모든 실패 분석은
이 입력들에서만 파생된다.

데이터 보강, 수정, 가공은
Core 5 이후 단계에서 금지된다.
""")


[Core 5 Input Freeze Complete]

이 노트북은 Core 2~4의 산출물을
'있는 그대로' 불러왔으며,
이후 모든 실패 분석은
이 입력들에서만 파생된다.

데이터 보강, 수정, 가공은
Core 5 이후 단계에서 금지된다.

